In [ ]:
import pandas as pd
import openpyxl

In [ ]:
priorities_df = pd.read_json('priorities.json')

In [ ]:
companies = {'Zeix': 20, 'SCS Supercomputing Systems': 20, 'Kyburz': 20, 'Siemens Mobility AG': 20, 'Varian': 20, 
             'MAN Energy Solutions':20, 'Universitätsspital Zürich': 20, 'Acht Grad Ost': 20, 'Weidmann': 20, 
             'True Wealth': 20, 'PartnerRe': 20, 'Sensirion': 20, 'Hocoma': 20}
uni1 = {'Architektur (ETH)': 30, 'Biologie (ETH)': 60, 'Raumbezogene Ingenieurwissenschaften (ETH)': 16, 
        'Umweltingenieurwissenschaften (ETH)': 16, 'Autonome Agile Drohnen (UZH)': 40, 
        'Biodiversität in Zeit und Raum (UZH)': 24, 'Einblick in aktuelle Forschung am Physik-Institut (UZH)': 20}
uni2 = {'Chemie (ETH)': 25, 'Materialwissenschaft (ETH)': 20, 'Medizin studieren an der ETH': 50, 
        'Pharmazeutische Wissenschaften (ETH)': 24, 'Physik (ETH)': 20, 'Illuminating the Chemistry of Life (UZH)': 30, 
        'Was ist ein Programm? (UZH)': 20}
uni3 = {'Aktuelle Forschung am CERN (UZH)': 30, 'Hyperwürfel (UZH)': 20, 'Krebsforschung (UZH)': 20, 
        'Informatik (ETH)': 16, 'Informationstechnologie und Elektrotechnik (ETH)': 30, 
        'Maschineningenieurwissenschaften (ETH)': 40, 'Mathematik (ETH)': 30}
workshops = {'Oberflächen und Farben': 20, 'Fourier-Reihen': 20, 'Datenexploration': 20, 
             'Astrophysik, Planetologie und Raumfahrt': 20, 'Molekulare Schalter – eine on/off-Beziehung': 12, 
             "Let's Arduino": 12, 'Die Hoffnung ist erneuerbar': 20, "Swiss Young Physicists' Tournament": 20, 
             'Von Big Data und Künstlicher Intelligenz zur Umweltinformatik': 16, 
             'Woher kommst du wirklich? Frag deine DNA!': 20, 'Astronomie': 20, 
             'Entsorgung, Recycling und Abwasserreinigung in Zürich': 16, 
             'Praktische Molekulargenetik: Wolbachia – auf der Suche nach Bakterien-DNA in selbstgefangenen Insekt': 14, 
             'Energiespeicherung – Vom Akku bis zum Pump-Speicherstausee': 20}

blocks = {'companies': companies, 'uni1': uni1, 'uni2': uni2, 'uni3': uni3, 'workshops': workshops}
block_names = [block for block in blocks.keys()]

In [ ]:
dict = {}
first = {}
seconds = {}
thirds = {}
options = {}
max_length_first = {}
max_length_seconds = {}
max_length_thirds = {}

last_name_list = []
first_name_list = []
class_list = []

choices_df = pd.DataFrame()


for block_name in block_names:
    first[block_name] = []
    seconds[block_name] = []
    thirds[block_name] = []
    max_length_first[block_name] = 0
    max_length_seconds[block_name] = 0
    max_length_thirds[block_name] = 0

    options[block_name] = []

for id, row in priorities_df.iterrows():
    last_name_list.append(row['last_name'])
    first_name_list.append(row['first_name'])
    class_list.append(row['class'])

    for block_id, block_name in enumerate(block_names):

        p1 = []
        p2 = []
        p3 = []

        for item in row['priorities']:
            if block_id == item['module_type'] - 1:
                if item['priority'] == 3:
                    p1.append(item['module_name'])
                if item['priority'] == 2:
                    p2.append(item['module_name'])
                if item['priority'] == 1:
                    p3.append(item['module_name'])

            if len(p1) > max_length_first[block_name]:
                max_length_first[block_name] = len(p1)
            if len(p2) > max_length_seconds[block_name]:
                max_length_seconds[block_name] = len(p2)
            if len(p3) > max_length_thirds[block_name]:
                max_length_thirds[block_name] = len(p3)
        
        first[block_name].append(p1)
        seconds[block_name].append(p2)
        thirds[block_name].append(p3)

        options[block_name].append(p1)
        options[block_name].append(p2)
        options[block_name].append(p3)


dict['last_name'] = last_name_list
dict['first_name'] = first_name_list
dict['class'] = class_list

options_flat = {}

for block_name in block_names:
    col_names_first = [f'{block_name}_first_{i}' for i in range(max_length_first[block_name])]
    col_names_seconds = [f'{block_name}_second_{i}' for i in range(max_length_seconds[block_name])]
    col_names_thirds = [f'{block_name}_third_{i}' for i in range(max_length_thirds[block_name])]

    dict[f'{block_name}_first_choice'] = first[block_name]    
    dict[f'{block_name}_second_choices'] = seconds[block_name]
    dict[f'{block_name}_third_choices'] = thirds[block_name]

    choices_df[col_names_first] = pd.DataFrame(dict[f'{block_name}_first_choice'], dtype='category')
    choices_df[col_names_seconds] = pd.DataFrame(dict[f'{block_name}_second_choices'], dtype='category')
    choices_df[col_names_thirds] = pd.DataFrame(dict[f'{block_name}_third_choices'], dtype='category')

    options_flat[block_name] = set([item for sublist in options[block_name] for item in sublist])

In [ ]:
choices_df.head()

In [ ]:
weights = [3, 2, 1]

n_people = len(priorities_df)

block_names = [block for block in blocks.keys()]
id_list = [f'ID{i:03}' for i in range(n_people)]

dict = {}
first = {}
seconds = {}
thirds = {}
score = {}

max_length_first = {}
max_length_seconds = {}
max_length_thirds = {}

last_name_list = []
first_name_list = []
class_list = []

choices_df = pd.DataFrame()


for block_name in block_names:
    first[block_name] = []
    seconds[block_name] = []
    thirds[block_name] = []
    score[block_name] = []
    max_length_first[block_name] = 0
    max_length_seconds[block_name] = 0
    max_length_thirds[block_name] = 0

for id, row in priorities_df.iterrows():
    last_name_list.append(row['last_name'])
    first_name_list.append(row['first_name'])
    class_list.append(row['class'])

    for block_id, block_name in enumerate(block_names):

        p1 = []
        p2 = []
        p3 = []

        for item in row['priorities']:
            if block_id == item['module_type'] - 1:
                if item['priority'] == 3:
                    p1.append(item['module_name'])
                if item['priority'] == 2:
                    p2.append(item['module_name'])
                if item['priority'] == 1:
                    p3.append(item['module_name'])

            if len(p1) > max_length_first[block_name]:
                max_length_first[block_name] = len(p1)
            if len(p2) > max_length_seconds[block_name]:
                max_length_seconds[block_name] = len(p2)
            if len(p3) > max_length_thirds[block_name]:
                max_length_thirds[block_name] = len(p3)
        
        first[block_name].append(p1)
        seconds[block_name].append(p2)
        thirds[block_name].append(p3)
        score[block_name].append(weights[0] * len(p1) + weights[1] * len(p2) + weights[2] * len(p3))


dict['last_name'] = last_name_list
dict['first_name'] = first_name_list
dict['class'] = class_list

total_score = [0] * n_people

for block_name in block_names:
    col_names_first = [f'{block_name}_first_{i}' for i in range(max_length_first[block_name])]
    col_names_seconds = [f'{block_name}_second_{i}' for i in range(max_length_seconds[block_name])]
    col_names_thirds = [f'{block_name}_third_{i}' for i in range(max_length_thirds[block_name])]
    col_name_score = f'{block_name}_score'

    dict[f'{block_name}_first_choice'] = first[block_name]    
    dict[f'{block_name}_second_choices'] = seconds[block_name]
    dict[f'{block_name}_third_choices'] = thirds[block_name]
    dict[f'{block_name}_score'] = score[block_name]

    choices_df[col_names_first] = pd.DataFrame(dict[f'{block_name}_first_choice'])
    choices_df[col_names_seconds] = pd.DataFrame(dict[f'{block_name}_second_choices'])
    choices_df[col_names_thirds] = pd.DataFrame(dict[f'{block_name}_third_choices'])
    choices_df[col_name_score] = pd.DataFrame(dict[f'{block_name}_score'])

    for i, s in enumerate(dict[f'{block_name}_score']):
        total_score[i] += s

dict['total_score'] = total_score
choices_df['total_score'] = pd.DataFrame(total_score)


In [ ]:
choices_df.head()

In [ ]:
filter_col = [col for col in choices_df if col.startswith('uni1')]
(choices_df.iloc[111])[filter_col]